## Part3: Adding Memory to the chatbot
When the chatbot can't remember the context of previous interactios, this limits its ability to have coherent, multi-turn conversations.
Langgraph solves this problem through **persistent checkpointer**. If you provide a checkpointer when compiling the graph and a thread_id when callin your graph, LangGraph automatically sves the state after each step. When you invoke the graph again using the same thread_id, the graph loads its saved state, allowing the chatbot to pick up where it left off.

We will see later that checkpointing is much more powerful than simple chat memory - it lets you save and resume complex state at any time for error recovery, human-in-the-loop workflows, time travel interactions, and more. But before we get too ahead of ourselves, let's add checkpointing to enable multi-turn conversations

In [4]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


In [ ]:
# We replace BasicToolNode with LnagGraph's prebuilt ToolNode and tools_condition
# because these do some nice things like parallel API execution.

In [5]:
from typing import Annotated
from langchain_google_vertexai import ChatVertexAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from configuracion import TAVILY_API_KEY, MODEL_GEMINI, GCP_PROJECT_ID, GCP_REGION, scoped_creds

from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tavilySearchAPIWrapper = TavilySearchAPIWrapper(tavily_api_key=TAVILY_API_KEY)
tool = TavilySearchResults(max_results=2, api_wrapper=tavilySearchAPIWrapper)
tools = [tool]
llm = ChatVertexAI(
                model_name=MODEL_GEMINI,
                project=GCP_PROJECT_ID,
                location=GCP_REGION,
                credentials=scoped_creds,
                max_output_tokens=8192,
                temperature=0.2,
        )
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)
#Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

#Finally, compile the graph with the provided checkpointer
graph = graph_builder.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "1"}}
user_input = "busca en internet noticias sobre mexico?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

busca en internet noticias sobre mexico?
================================== Ai Message ==================================

Lo siento, no tengo acceso a internet para buscar noticias en tiempo real.  Para obtener información actualizada sobre México, te recomiendo visitar sitios web de noticias confiables como:

* **El Universal:** https://www.eluniversal.com.mx/
* **Milenio:** https://www.milenio.com/
* **La Jornada:** https://www.jornada.com.mx/
* **CNN en Español:** https://cnnespanol.cnn.com/

¡Espero que esto te ayude!
